In [5]:
from datetime import datetime
import numpy as np
import pandas as pd
import pandas_datareader.data as web
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
# importing os module for environment variables
import os
# importing necessary functions from dotenv library
from dotenv import load_dotenv, dotenv_values 

In [6]:

# loading variables from .env file
load_dotenv() 
 
# accessing and printing value
ALPHAVANTAGE_API_KEY = os.getenv("ALPHAVANTAGE_API_KEY")
print(os.getenv("ALPHAVANTAGE_API_KEY"))

SGRQ1JQEJ152DE6W


In [7]:
df = web.DataReader("snd", "av-daily", start=datetime(2016, 1, 1), end=datetime.now(), api_key=ALPHAVANTAGE_API_KEY)

In [8]:
df['date'] = pd.to_datetime(df.index) # df.index
df['year'] = df['date'].dt.year
df['quarter'] = df['date'].dt.quarter
df['month'] = df['date'].dt.month
df.set_index(['year', 'quarter', 'month', 'date'], inplace=True)
df['date'] = df.index.get_level_values('date')
df.head(10)

open     high    low  close   volume  \
year quarter month date                                                
2016 4       11    2016-11-04  10.30  11.0700  10.30  10.99  3419720   
                   2016-11-07  11.03  11.7500  10.99  11.07   599717   
                   2016-11-08  11.10  11.2500  10.90  11.07   267382   
                   2016-11-09  11.00  12.0500  11.00  11.58   270679   
                   2016-11-10  11.41  11.8500  11.27  11.57   287974   
                   2016-11-11  11.50  11.5575  11.33  11.36    43952   
                   2016-11-14  11.35  11.4000  10.93  11.18    80816   
                   2016-11-15  11.15  11.7955  11.14  11.20    83626   
                   2016-11-16  11.06  11.1450  10.89  10.98   232328   
                   2016-11-17  10.96  11.1499  10.75  10.97    43369   

                                    date  
year quarter month date                   
2016 4       11    2016-11-04 2016-11-04  
                   2016-11-07 2016-11-07  
                   2016-11-08 2016-11-08  
                   2016-11-09 2016-11-09  
                   2016-11-10 2016-11-10  
                   2016-11-11 2016-11-11  
                   2016-11-14 2016-11-14  
                   2016-11-15 2016-11-15  
                   2016-11-16 2016-11-16  
                   2016-11-17 2016-11-17

In [9]:
quarterly_data = df.groupby(['year', 'quarter'])[['date','open', 'high', 'low', 'close']].agg({
    'date': 'min',
    'open': 'first',
    'high': 'max',
    'low': 'min',
    'close': 'last'
}).reset_index()
mothly_data = df.groupby(['year', 'quarter', 'month'])[['date','open', 'high', 'low', 'close']].agg({
    'date': 'min',
    'open': 'first',
    'high': 'max',
    'low': 'min',
    'close': 'last'
}).reset_index()

quarterly_data['date'] = quarterly_data['date'].apply(lambda x: x.replace(day=1))
mothly_data['date'] = mothly_data['date'].apply(lambda x: x.replace(day=1))


In [10]:
mothly_data.head(20)

,year,quarter,month,date,open,high,low,close
0,2016,4,11,2016-11-01,10.30,14.350,10.300,13.67
1,2016,4,12,2016-12-01,13.80,17.000,12.650,16.55
2,2017,1,1,2017-01-01,16.75,19.840,16.205,17.40
3,2017,1,2,2017-02-01,16.89,21.990,16.340,17.17
4,2017,1,3,2017-03-01,17.39,18.719,12.510,16.25
5,2017,2,4,2017-04-01,16.29,16.670,12.160,12.18
6,2017,2,5,2017-05-01,12.17,13.700,9.040,9.74
7,2017,2,6,2017-06-01,9.86,10.410,7.510,8.91
8,2017,3,7,2017-07-01,8.94,9.200,5.900,6.58
9,2017,3,8,2017-08-01,6.40,6.920,4.810,5.97


In [11]:
def prepend_statement_to_column_name(columns, statement):
  cols = {}
  for idx, col in enumerate(columns): 
    cols.update({col: statement + ' ' + col})
  
  return cols

In [12]:
bsh = pd.read_csv('../data/snd-bs-q-20-12.csv', sep='\t', encoding="latin-1", index_col=0, dtype=str).T
ist = pd.read_csv('../data/snd-is-q-20-12.csv', sep='\t', encoding="latin-1", index_col=0, dtype=str).T
cfs = pd.read_csv('../data/snd-cf-q-20-12.csv', sep='\t', encoding="latin-1", index_col=0, dtype=str).T
# cols = df.iloc[0]
# df[0:][5:]
# df.dropna(inplace = True)  

bsh.rename(columns=prepend_statement_to_column_name(bsh.columns, "bs"), inplace=True)
ist.rename(columns=prepend_statement_to_column_name(ist.columns, "is"), inplace=True)
cfs.rename(columns=prepend_statement_to_column_name(cfs.columns, "cf"), inplace=True)

# col = pd.to_datetime(bsh.index, format='%d.%m.%Y', dayfirst=True)
col = pd.to_datetime(bsh.index, format='mixed', dayfirst=True)
bsh.set_index(col, inplace=True)
bsh['date'] = col


# bsh.rename(columns=cl, inplace=True)

# bsh.columns[:]
# col = pd.to_datetime(ist.index, format='%d.%m.%Y', dayfirst=True)
col = pd.to_datetime(ist.index, format='mixed', dayfirst=True)
ist['date'] = col
ist.set_index(col, inplace=True)


# col = pd.to_datetime(cfs.index, format='%d.%m.%Y', dayfirst=True)
col = pd.to_datetime(cfs.index, format='mixed', dayfirst=True)
cfs.set_index(col, inplace=True)
cfs['date'] = col
# ist['year'] = col.year
# ist[ 'quarter'] = col.quarter
# ist[ 'month'] = col.month
# ist.columns[:]
# ist[['Revenues', 'Cost of Goods Sold', 'Selling General & Admin Expenses', 'Interest Expense']][0:10]

merged = pd.merge(bsh, ist, on='date', how='outer', validate='1:m', sort=True)
merged = pd.merge(merged, cfs, on='date', how='outer', validate='1:m', sort=True)
# merged.columns[:]

merged.insert(0, 'month', col.month)
merged.insert(0, 'quarter', col.quarter)
merged.insert(0, 'year', col.year)
col = merged.pop('date')
merged.insert(0, 'date', col)

# merged.dtypes[['Change In Accounts Receivable', 'Cost of Goods Sold', 'Selling General & Admin Expenses', 'Interest Expense']]


In [13]:
def move_column_to_front(df, col_to_move):
    first_column = df.pop(col_to_move)
    df.insert(0, col_to_move, first_column) 
    

In [14]:
# merged.to_csv('../data/merged.csv', index=True, encoding='utf-8', sep='\t')

merged[:10]


,date,year,quarter,month,bs Cash And Equivalents,bs Total Cash And Short Term Investments,bs Accounts Receivable,bs Other Receivables,bs Total Receivables,bs Inventory,...,cf Net Change in Cash,cf Supplementary Data:,cf Free Cash Flow,cf % Change YoY,cf % Free Cash Flow Margins,"cf Cash and Cash Equivalents, Beginning of Period","cf Cash and Cash Equivalents, End of Period",cf Cash Interest Paid,cf Cash Taxes Paid,cf Cash Flow per Share
0,2016-12-31,2016,4,12,"46,56","46,56","5,74","0,03","5,77","10,34",...,"46,82",NaN,"18,14","71,5 %","61,6 %","3,90","47,53","1,03","0,72","0,59"
1,2017-03-31,2017,1,3,"72,67","72,67","10,80","0,24","11,04","9,08",...,"26,11",NaN,"2,22","2,3 %","8,9 %","47,53","73,64","0,02","0,03","0,06"
2,2017-06-30,2017,2,6,"63,36","63,36","14,96","0,49","15,46","8,81",...,"(9,80)",NaN,"(9,29)","(396,0 %)","(31,2 %)","47,53","63,84","0,08","7,57","(0,23)"
3,2017-09-30,2017,3,9,"51,33","51,33","21,53","0,74","22,27","7,67",...,"(12,03)",NaN,"(11,96)","(1 733,6 %)","(30,4 %)","47,53","51,82","0,05","0,06","(0,30)"
4,2017-12-31,2017,4,12,"34,74","34,74","24,57","0,80","24,57","9,53",...,"(16,59)",NaN,"(16,51)","(191,0 %)","(38,4 %)","47,53","35,23","0,11","0,01","(0,41)"
5,2018-03-31,2018,1,3,"2,14","2,14","27,90","1,60","29,50","5,27",...,"(32,61)",NaN,"(48,48)","(2 282,8 %)","(113,7 %)","35,23","2,62","0,10","0,21","(1,20)"
6,2018-06-30,2018,2,6,"1,75","1,75","28,99","1,10","30,09","11,55",...,"(0,88)",NaN,"0,55","106,0 %","1,0 %","35,23","1,75","0,56","0,45","0,01"
7,2018-09-30,2018,3,9,"1,19","1,19","31,09","1,10","32,19","16,40",...,"(0,56)",NaN,"(0,00)","100,0 %","(0,0 %)","35,23","1,19","0,22","0,07","(0,00)"
8,2018-12-31,2018,4,12,"1,47","1,47","26,81",NaN,"26,81","18,58",...,"0,28",NaN,"2,75","116,7 %","5,3 %","35,23","1,47","1,66","0,67","0,07"
9,2019-03-31,2019,1,3,"2,37","2,37","39,52",NaN,"39,52","13,57",...,"0,90",NaN,"(2,18)","95,5 %","(4,2 %)","1,47","2,37",NaN,NaN,"(0,05)"


In [15]:
import re

def normalize_string(s):
    """
    Normalize a string by converting to lowercase, trimming, replacing non-alphanumeric characters with underscores,
    replacing spaces with underscores, and collapsing multiple consecutive underscores into a single underscore.
    
    :param s: The input string
    :return: The normalized string
    """
    s = s.lower()  # convert to lowercase
    s = s.strip()  # trim from both sides
    s = re.sub(r'[^a-zA-Z0-9\s%&]', '_', s)  # replace non-alphanumeric characters with underscores
    s = re.sub(r'\s', '_', s)  # replace spaces with underscores
    s = s.replace("&", "_and_") # replace '&' with '_and_'
    # Move '%' to the end of the string and prepend an underscore
    if '%' in s:
        s = s.replace('%', '')  # remove '%' from the string                
        s = s + '_%'  # append '_%' to the end of the string    
                
    s = s.strip('_')  # trim underscores from the start and end of the string
    s = re.sub(r'_+', '_', s)  # collapse multiple consecutive underscores into a single underscore
    return s

In [16]:
# merged[['date', 'Revenues', 'EBITDA', 'Price Close']][0:10]
# merged['Revenues'] = pd.to_numeric(merged['Revenues'], errors='coerce')
row = []
cols = {}
for idx, col in enumerate(merged.columns): 
  row.append([col, normalize_string(col)])
  cols.update({col: normalize_string(col)})
  # print(col, normalize_string(col))

ndf = merged.rename(columns=cols, inplace=False)
# dict.fromkeys(row[0:], row[1:])
# print (merged.columns)
ndf[1:10]
# Export the column names to a CSV file
pd.DataFrame(row).to_csv('../data/column_names.csv', columns=[0, 1], index=False)



In [17]:
import re
def is_float(value):
    try:
        float(value)
        return True
    except ValueError:
        return False
      
def normalize_number(val:str):
  if pd.isna(val):
    # return not a number
    return np.nan 
  if isinstance(val, (datetime)) or isinstance(val, int) or isinstance(val, float):
    return val
  
  is_procent = False
  is_negative = False
  val = val.strip().replace(' ', '')
  if '%' in val:
    is_procent = True
    val = val.replace('%', '')    
  if '(' in val:
    is_negative = True
    val = val.replace('(', '')
    val = val.replace(')', '')    
  
  val = val.replace(',', '.')  
  if not is_float(val):
    return val
  
  val = float(val)
  if is_procent:
    val = val / 100
  if is_negative:
    val = -val
  if val =='':
    return np.nan
  return val

In [21]:
# Change all columns to the appropriate types
# merged = pd.to_numeric(merged, errors='coerce')
# ndf.columns[:]

cols_dupl = [col for col in ndf.columns if col.endswith('_y')]
# cols_dupl[:]
# col = ['date', 'supplementary_data_y', 'net_income_y', 'depreciation_and_amortization_y'] + ['supplementary_data_x', 'net_income_x', 'depreciation_and_amortization_x']

qndf = ndf.loc[:, ~ndf.columns.duplicated(keep='first')]


# nndf = qndf['bs_comprehensive_income_and_other'].map(normalize_number)
nndf = qndf.map(normalize_number)

# ctdf = convert_to_types(ndf)
""" for idx, col in enumerate(ndf.columns):
    print(f"Column {idx}: {col}")
    print("_"*60, "\n", ndf.iloc[:, idx])  """   
# ctdf = ndf.str.replace(',', '.').astype(float)
# ctdf = convert_to_types(ndf) # .convert_dtypes()
# res = ndf.columns[qndf.columns.str.contains("net_income")]
# res_df = ctdf[res]

# qndf[:10]
# print ('_'*100)
# nndf[['date', 'net_income'], :10]
# for idx, col in enumerate(nndf.columns):
    # if 'is_' in col: 
        # print(f"Column {idx}: {col}")
        # print("_"*60, "\n", nndf.iloc[:, idx])
# res_df = ctdf[res]
# nndf['bs_comprehensive_income_and_other'][20]
qndf['cf_change_in_accounts_receivable']


0      (2,71)
1      (5,05)
2      (4,17)
3      (6,57)
4      (3,04)
5      (3,33)
6      (0,98)
7      (2,10)
8        4,24
9     (12,70)
10    (13,45)
11     (6,98)
12     (4,88)
13       5,63
14       2,04
15    (19,19)
16       5,44
17       2,97
18      35,69
19     (4,98)
20     (2,79)
21     (8,81)
22     (8,80)
23       2,78
24     (1,63)
25     (1,44)
26       7,01
27       6,07
28     (1,61)
29    (11,98)
30       7,21
Name: cf_change_in_accounts_receivable, dtype: object

In [20]:
nndf.to_csv('../data/nndf.csv', index=False, encoding='utf-8', sep='\t', decimal=',')

# type(nndf['bs_cash_and_equivalents'][10])

In [ ]:
# vykresleni korelacniho diagramu pro dvojici vybranych atributu
# první sloupec: x-ové souřadnice
# druhý sloupec: y-ové souřadnice
# 
# plt.title("Classes")
# vykresleni mrizky korelacnich diagramu
fig, axes = plt.subplots(nrows=4, ncols=4)

# rozmery grafu
fig.set_figheight(15)
fig.set_figwidth(15)
ist[['Revenues', 'Cost of Goods Sold']]

feature_names = ['Revenues', 'Cost of Goods Sold', 'Selling General & Admin Expenses', 'Interest Expense']
target = ist['Net Income'].to_numpy()
data = ist[feature_names].to_numpy()

# vyplneni mrizky
for row in range(4):
    for column in range(4):
        ax = axes[row][column]
        if row == column:
            # na diagonale jsou prazdna mista
            fig.delaxes(ax)
            continue
        # pridat korelacni diagram do mrizky
        print(f"row: {row}, col: {column}, {data[:, row]}, {target}")
        scatter = ax.scatter(data[:, row], data[:, column], c=target)
        # popisky os
        ax.set(xlabel=feature_names[row], ylabel=feature_names[column])
 
 
# zbavit se prazdneho mista okolo bunek mrizky
plt.tight_layout()

# ulozeni diagramu do souboru
plt.savefig("52.png")
 
# zobrazeni diagramu
plt.show()


In [ ]:


# vyplneni mrizky
for row in range(4):
    for column in range(4):
        ax = axes[row][column]
        if row == column:
            # na diagonale jsou prazdna mista
            fig.delaxes(ax)
            continue
        # pridat korelacni diagram do mrizky
        scatter = ax.scatter(data[:, row], data[:, column], c=target)
        # popisky os
        ax.set(xlabel=feature_names[row], ylabel=feature_names[column])
 
 
# zbavit se prazdneho mista okolo bunek mrizky
plt.tight_layout()

# ulozeni diagramu do souboru
plt.savefig("52.png")
 
# zobrazeni diagramu
plt.show()